# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 28 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [132]:
!pip install numpy torch torchsummaryX wandb

In [133]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
import torch.multiprocessing as mp
import torch.nn as nn
from tqdm.auto import tqdm
import time
import random
import os
import datetime
import wandb
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_processes = 4
print("Device: ", device)

Device:  cpu


In [134]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
# from google.colab import drive
# drive.mount('/content/drive')

In [135]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [136]:
# !pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
# !mkdir /root/.kaggle
# !touch /root/.kaggle/kaggle.json
# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"suriyaganesh","key":"74848730a16663d8aca7f6ebdeacb10f"}')
#     # Put your kaggle username & key here

# !chmod 600 /root/.kaggle/kaggle.json

In [137]:
!ls /kaggle/input/

11785-hw1p2-f23


In [138]:
# # commands to download data from kaggle

# !kaggle competitions download -c 11785-hw1p2-f23
# !mkdir '/content/data'

# !unzip -qo /content/11785-hw1p2-f23.zip -d '/content/data'

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [139]:
# from numpy.core.overrides import verify_matching_signatures
# import numpy as np
# # Create a list of matrices (replace these with your actual matrices)
# matrix_list = [np.array([[1, 2], [3, 4]]), np.array([[5, 6], [7, 8]]), np.array([[9, 10], [11, 12]])]

# # Concatenate matrices horizontally (along columns)
# horizontal_concatenated = np.concatenate(matrix_list, axis=1)

# # Concatenate matrices vertically (along rows)
# vertical_concatenated = np.concatenate(matrix_list, axis=0)

# # Print the results
# print("Horizontally concatenated:")
# print(horizontal_concatenated)

# print("Vertically concatenated:")
# print(vertical_concatenated)
# print(np.concatenate(vertical_concatenated))

# print(matrix_list)

# mfcc_path = "/kaggle/input/11785-hw1p2-f23/11-785-f23-hw1p2/test-clean/mfcc/"
# # transcript_path = "/kaggle/input/11-785-f23-hw1p2/dev-clean/transcript/84-121123-0001.npy"
# mfccnp = []
# for i in os.listdir(mfcc_path):
#   mfccnp.append(np.load(mfcc_path+i))

# # transcript_path = np.load(transcript_path)
# # print(transcript_path.shape)
# mfccconcat = np.concatenate(mfccnp)
# mfccpad = np.pad(mfccconcat, ((1, 1), (0, 0)), 'constant', constant_values=0)
# print(mfccconcat.shape)
# print(mfccpad.shape)
# print(mfccpad[1934139])

In [140]:
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, val=True,phonemes=PHONEMES, context=0,
                 partition="train-clean-100"):  # Feel free to add more arguments

        self.context = context
        self.phonemes = phonemes

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir = root + "mfcc/"
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = root + "transcript/"

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names = sorted(os.listdir(self.mfcc_dir))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names = sorted(os.listdir(self.transcript_dir))

#         c = list(zip(mfcc_names, transcript_names))

#         random.shuffle(c)

#         mfcc_names, transcript_names = zip(*c)

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        if val:
            mfcc_names, transcript_names = (mfcc_names[:int(len(mfcc_names))], transcript_names[:int(len(transcript_names))])
        else:
            mfcc_names, transcript_names = (mfcc_names[:int(len(mfcc_names) / 10)], transcript_names[:int(len(transcript_names) / 10)])
        # mfcc_names, transcript_names =  (mfcc_names[20:30],transcript_names[20:30])

        self.mfccs, self.transcripts = [], []

        # TODO: Iterate through mfccs and transcripts
        for m, t in zip(mfcc_names, transcript_names):
            #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir + m) 
            mfcc = (mfcc - np.mean(mfcc, axis=0))/np.std(mfcc, axis=0)
            #   Do Cepstral Normalization of mfcc (explained in writeup)
            #   Load the corresponding transcript
            transcript = np.load(self.transcript_dir + t)
            transcript = transcript[1:-1]
            # Remove [SOS] and [EOS] from the transcript
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
            #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        # NOTE:
        # Each mfcc is of shape T1 x 28, T2 x 28, ...
        # Each transcript is of shape (T1+2), (T2+2),... before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 28 (Where T = T1 + T2 + ...)
        self.mfccs = np.concatenate(self.mfccs, axis=0)

        # TODO: Concatenate all transcripts in self.transcripts such that
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts, axis=0)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)
        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), 'constant', constant_values=0)

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers

#         print("!!", set(self.transcripts))
        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        self.transcripts = [PHONEMES.index(transcript) for transcript in self.transcripts]
        print(len(self.mfccs), len(self.transcripts))

    #         print("!",self.transcripts)
    #         print("!",self.transcripts)
    # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        phonemes = torch.tensor(self.transcripts[ind]).to(device)
        ind = ind + self.context
        # print(ind-self.context,ind+self.context+1)
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[ind - self.context: ind + self.context + 1].flatten()

        frames = torch.FloatTensor(frames).to(device)  # Convert to tensors
        #         phonemes    = self.transcripts[ind]

        return frames, phonemes



In [141]:
class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes

        self.mfcc_dir       = root+"mfcc/"
        mfcc_names          = os.listdir(self.mfcc_dir)


        # Making sure that we have the same no. of mfcc and transcripts

        self.mfccs = []

        # TODO: Iterate through mfccs and transcripts
        for m in mfcc_names:

        #   Load a single mfcc
            mfcc        = np.load(self.mfcc_dir+m)
            self.mfccs.append(mfcc)
        self.mfccs          = np.concatenate(self.mfccs, axis=0)
        self.length = len(self.mfccs)
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), 'constant', constant_values=0)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        ind = ind+self.context
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[ind-self.context: ind+self.context+1].flatten()
        frames      = torch.FloatTensor(frames).to(device) # Convert to tensors

        return frames

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

In [142]:
config = {
    'epochs'        : 15,
    'batch_size'    : 1024,
    'context'       : 35,
    'init_lr'       : 1e-2,
    'lr_decay':0.0,
    'architecture'  : 'very-low-cutoff',
    'randomSeed':2,
    'hiddenDims': [2048,1024,1024,1024,1024,512,256]
    }

# Create Datasets

In [143]:
#TODO: Create a dataset object using the AudioDataset class for the training data
train_data = AudioDataset("/kaggle/input/11785-hw1p2-f23/11-785-f23-hw1p2/train-clean-100/",context=config['context'], val=False)


3495464 3495394


In [144]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 0,
    batch_size  = config['batch_size'],
    pin_memory  = False,
    shuffle     = True
)

print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
# print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
# print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  1024
Context        :  35
Input size     :  1988
Output symbols :  42
Train dataset samples = 3495394, batches = 3414


In [145]:
a,b = train_data.__getitem__(20)
a.shape
# print(b.is_cuda)
torch.set_printoptions(edgeitems=3)
print(a.size())
print(b)
# print(a)
i = 0
for i, (a,b) in enumerate(train_data):
    pass
print(i)
print(train_data.__len__())
print(train_data.__getitem__(12495))


torch.Size([1988])
tensor(0)
3495393
3495394
(tensor([ 1.4057,  0.1865, -1.9150,  ...,  0.8905, -0.1089,  0.8735]), tensor(21))


In [146]:
#Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([1024, 1988]) torch.Size([1024])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [147]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(nn.Module):
    def __init__(self, sizeList):
        super(Network, self).__init__()
        layers = []
        self.sizeList = sizeList
         nn.Sequential(
        nn.Linear(in_features=1988, out_features=2048, bias=True)
        nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        nn.LeakyReLU(negative_slope=0.01)
        nn.Linear(in_features=2048, out_features=1024, bias=True)
        nn.BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        nn.LeakyReLU(negative_slope=0.01)
        nn.Linear(in_features=1024, out_features=1024, bias=True)
        nn.BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        nn.LeakyReLU(negative_slope=0.01)
        nn.Dropout(p=0.2, inplace=False)
        nn.Linear(in_features=1024, out_features=1024, bias=True)
        nn.BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        nn.LeakyReLU(negative_slope=0.01)
        nn.Linear(in_features=1024, out_features=1024, bias=True)
        nn.BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        nn.LeakyReLU(negative_slope=0.01)
        nn.Dropout(p=0.2, inplace=False)
        nn.Linear(in_features=1024, out_features=512, bias=True)
        nn.LeakyReLU(negative_slope=0.01)
        nn.Linear(in_features=512, out_features=256, bias=True)
        nn.LeakyReLU(negative_slope=0.01)
        nn.Linear(in_features=256, out_features=42, bias=True)
      )
        
    def forward(self, x):
        return self.net(x)

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [148]:
INPUT_SIZE  = (2*config['context'] + 1) * 28 # Why is this the case?
model       = Network([INPUT_SIZE] + config["hiddenDims"] + [len(PHONEMES)])
# frames, p = train_data.__getitem__(0)
print(model)
print(frames.shape)

# summary(model, frames)
# Check number of parameters of your network
# Remember, you are limited to 25 million parameters for HW1 (including ensembles)

Network(
  (net): Sequential(
    (0): Linear(in_features=1988, out_features=2048, bias=True)
    (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Linear(in_features=2048, out_features=1024, bias=True)
    (4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=1024, out_features=1024, bias=True)
    (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Dropout(p=0.2, inplace=False)
    (10): Linear(in_features=1024, out_features=1024, bias=True)
    (11): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): LeakyReLU(negative_slope=0.01)
    (13): Linear(in_features=1024, out_features=1024, bias=True)
    (14): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [149]:
torch.cuda.is_available()
model = model.to(device)

In [150]:
criterion = torch.nn.CrossEntropyLoss().to(device) # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.AdamW(model.parameters(), lr= config['init_lr']) #Defining Optimizer

# Recommended : Define Scheduler for Learning Rate,
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [151]:
torch.cuda.empty_cache()
gc.collect()

2235

In [152]:
def train(model, train_loader,  optimizer,criterion):
    model.train()
    running_loss = 0.0
    start_time = time.time()
    dropout = torch.nn.Dropout(p=0.5, inplace=False)
    acc = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data = data.to(device)
        target = target
        
        outputs = dropout(model(data))
        loss = criterion(outputs, target)
        acc += torch.sum(torch.argmax(outputs, dim= 1) == target).item()/outputs.shape[0]
        running_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print("Finished " + str(batch_idx) + "\t Timestamp: "+ str(time.time() - start_time))
            print("loss :", loss,"acc :", acc/(batch_idx+1))

    end_time = time.time()
    running_loss = running_loss / batch_idx
    acc = acc/batch_idx
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    return running_loss, acc

In [153]:
def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()
        start_time = time.time()
        
        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0
        for batch_idx, (data, target) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)
            
            outputs = model(data)
            
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()
            
            loss = criterion(outputs, target).detach()
            running_loss += loss.item()
            if batch_idx % 1000 == 0:
                print("Finished " + str(batch_idx) + "\t Timestamp: "+ str(time.time() - start_time))
        
        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        print('Testing Loss: ', running_loss)
        print('Testing Accuracy: ', acc, '%')
        return running_loss, acc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [154]:
wandb.login(key="apikey") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [155]:
# Create your wandb run
# run = wandb.init(
#     name    = "first-run", ### Wandb creates random run names if you skip this field, we recommend you give useful names
#     reinit  = True, ### Allows reinitalizing runs when you re-run this cell
#     #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
#     #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
#     project = "hw1p2", ### Project should be created in your wandb account
#     config  = config ### Wandb Config for your run
# )

In [156]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
# wandb.save('model_arch.txt')

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
# wandb.watch(model, log="all")

# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data= AudioDataset("/kaggle/input/11785-hw1p2-f23/11-785-f23-hw1p2/dev-clean/",context=config['context'], val=True)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 0,
    batch_size  = config['batch_size'],
    pin_memory  = False,
    shuffle     = False
)

for epoch in range(config['epochs']):
    

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))
    curr_lr                 = float(optimizer.param_groups[0]['lr'])
#     print(model,train_loader,optimizer, criterion, sep="###")
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    
    
#     model.share_memory()
#     processes = []
#     for rank in range(num_processes):
#         p = mp.Process(target=train, args=(model,))
#         p.start()
#         processes.append(p)
#     for p in processes:
#         p.join()


#     mp.set_start_method('spawn')
#     p = mp.Process(target=train, args=(model,train_loader,optimizer,criterion), daemon=True)
#     p.start()
#     p.join()
#     print("x =", x.item())
    
    
    val_loss, val_acc       = test_model(model, val_loader, criterion)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
#     wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
#                'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
torch.save(model.state_dict(), './model.model')
model.save(os.path.join(wandb.run.dir, "model.h5"))
### Finish your wandb run
run.finish()

1928274 1928204

Epoch 1/15
Finished 0	 Timestamp: 1.3941848278045654
loss : tensor(3.7473, grad_fn=<NllLossBackward0>) acc : 0.0224609375
Finished 1000	 Timestamp: 568.8796234130859
loss : tensor(2.5314, grad_fn=<NllLossBackward0>) acc : 0.33882718843656345
Finished 2000	 Timestamp: 1136.3880393505096
loss : tensor(2.4039, grad_fn=<NllLossBackward0>) acc : 0.3755744198213393
Finished 3000	 Timestamp: 1703.9138746261597
loss : tensor(2.3386, grad_fn=<NllLossBackward0>) acc : 0.3980535134121959
Training Loss:  2.544311793710422 Time:  1938.8547694683075 s
Finished 0	 Timestamp: 0.21741747856140137
Finished 1000	 Timestamp: 219.51888918876648
Testing Loss:  1.4894773649435387
Testing Accuracy:  61.00962346307755 %
	Train Acc 40.4991%	Train Loss 2.5443	 Learning Rate 0.0100000
	Val Acc 6100.9623%	Val Loss 1.4895

Epoch 2/15
Finished 0	 Timestamp: 1.1274447441101074
loss : tensor(2.3310, grad_fn=<NllLossBackward0>) acc : 0.44921875
Finished 1000	 Timestamp: 567.7061786651611
loss : tensor(

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval()  # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []
    predicted_phoneme = None

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode():  # TODO

        for i, mfccs in enumerate(test_loader):
            mfccs = mfccs.to(device)

            logits = model(mfccs)
            # print(logits.shape)
            l = torch.argmax(logits, dim=1)
            # print("!!!",len(l))
            # print("!!!",l)
            ### Get most likely predicted phoneme with argmax
            predicted_phoneme = [PHONEMES[int(i)] for i in l]
            # print(predicted_phoneme)
            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            # TODO
            test_predictions.extend(predicted_phoneme)
            # print("!!",PHONEMES[predicted_phonemes])


    return test_predictions

In [1]:
# print(model)

NameError: name 'model' is not defined

In [ ]:
# model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
model.load_state_dict(torch.load("./model.model"))

# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioTestDataset("/kaggle/input/11785-hw1p2-f23/11-785-f23-hw1p2/test-clean/", context=config['context'])
test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 0,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

predictions = test(model, test_loader)
print(test_loader.__len__())
# predicted_phoneme = []
# print(test_data.__getitem__(1888))

# # for i in range(0,1887):
# #   test_loader.__next__()

# # print(test_loader.__next__())

# # while True:
# #     (x,y) = next(test_loader)
# #     print(y.shape)

# # for i in test_loader:
# #   print(i)


# for mfccs in test_loader:
#   print(mfccs.shape)
#   mfccs = mfccs.to(device)
#   logits = model(mfccs)
#   print(logits.shape)
#   ## Get most likely predicted phoneme with argmax
#   predicted_phoneme.append(torch.argmax(torch.argmax(logits, dim=1)))
#   break

# print(predicted_phoneme)
# # print(i,mfccs)
# print(mfccs.shape)



In [ ]:
print(train_data.__getitem__(250))
# print(predictions)
print(len(predictions))
# print(len(list(predictions)))
# print(torch.argmax(predictions))
# print(PHONEMES[torch.argmax(predictions)])
# print(PHONEM)

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
!pwd

In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11785-hw1p2-f23 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle